In [11]:
#Load packages
from netCDF4 import Dataset
import numpy as np
import matplotlib.colors as colors
import matplotlib.pyplot as plt
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
import sys
import netCDF4 as nc4
from torch import nn
from torch import optim
import progressbar
import pytorch_lightning as pl
import torch.nn.functional as F
from torchvision import transforms
from torch import nn
from torch import optim
import progressbar
sys.path.append("/home2/datahome/tpicard/python/Python_Modules_p3_pyticles/")
sys.path.append("../TRAINING/")
sys.path.append("../ANALYSE/")
import torch
from torch.utils.data import DataLoader, Dataset
from CNN_tools import *
from CNN_UNET import *
from pytorch_lightning.callbacks import ModelCheckpoint
from DATALOADER import Pdf_Image_DataSet
from variables import *
from tools_analyse import *
import importlib

list_rand = np.array([1225,  774 ,  1210,  
                  1374, 1359,  715,  804, 
                  1006, 148,  264,  1216,  339,  
                  279, 19, 82, 818,   74, 
                  230, 1352,  652, 1091,  
                  310, 730, 116,   60,  829])



# MASSE CENTER MOVEMENT
def masse_center_mov(pdf):
    mx = np.sum(np.multiply(lon,np.sum(pdf,axis=0)))
    my = np.sum(np.multiply(lat,np.sum(pdf,axis=1)))
    return np.sqrt(mx**2 + my**2)

def masse_center_mov_mxy(pdf):
    mx = np.sum(np.multiply(lon,np.sum(pdf,axis=0)))
    my = np.sum(np.multiply(lat,np.sum(pdf,axis=1)))
    return mx,my

# VARIANCE
def variance(pdf):
    D = 0
    mx,my = masse_center_mov_mxy(pdf)
    for i in range(100):
        for j in range(100):
            D = D + (500/499)*pdf[i,j]*((lon[i]-mx)**2+(lat[j]-my)**2)
    return D

lon = np.linspace(-400,400,nb_dx)
lat = np.linspace(-400,400,nb_dx)

# VARIANCE
def entropie(pdf):
    D = 0
    for i in range(100):
        for j in range(100):
            if pdf[i,j]==0:
                 continue
            else:
                D = D - pdf[i,j]*np.log(pdf[i,j])
    return D

# LOAD DATA FROM

In [12]:
#image_test = load_images_simu2('cinquieme')

sys.path.append("/home2/datahome/tpicard/python/Python_Modules_p3_pyticles/")
from Modules import *
tstart = 1900
my_simul = 'aperitif_simu2'
str_para = ' [{0},{1},{2},{3},[1,100,1]] '.format(jc-260,jc+260,ic-260,ic+260)
parameters = my_simul +str_para+ format(tstart)
simul = load(simul = parameters, floattype=np.float64, output=False)
f = simul.f

nc = nc4.Dataset('../DATA/vrt_ow_strain_mean_100x100_surf_j20.nc','r')
vrt_mean = np.asfortranarray(nc.variables['vrt'][::5])
ow_mean = np.asfortranarray(nc.variables['ow'][::5])/(np.mean(f)**2)
nc.close()

vrt_mean = vrt_mean[list_rand]
ow_mean = ow_mean[list_rand]

pdf_filter_test = load_pdf_filter_simu2()
pdf_filter_test = np.transpose(pdf_filter_test,(0,2,1,3,4))
pdf_filter_test = pdf_filter_test.reshape(pdf_filter_test.shape[0]*pdf_filter_test.shape[1],8,100,100)
pdf_filter_test = pdf_filter_test[::5]
pdf_filter_test = pdf_filter_test[list_rand,:]

args (['aperitif_simu2', '[357,877,1308,1828,[1,100,1]]', '1900'],)
args[0] ['aperitif_simu2', '[357,877,1308,1828,[1,100,1]]', '1900']
len(args[0]) 3
Loading filtered pdf simu2 ...
Pdf filtered simu2 loaded


# Compute score 

In [13]:

#LOAD super model
nc = nc4.Dataset('../DATA/supermodel_4L_2.nc','r')
super_model_4L = np.asfortranarray(nc.variables['super_model_norm'][list_rand,:])
nc.close()

#LOAD super model
nc = nc4.Dataset('../DATA/supermodel_surf_2.nc','r')
super_model_surf = np.asfortranarray(nc.variables['super_model_norm'][list_rand,:])
nc.close()

list_bhatta = np.zeros(pdf_filter_test.shape[0])
for i in range(pdf_filter_test.shape[0]):
    list_bhatta[i] = 1 - np.sum(np.sqrt(np.abs((np.multiply(super_model_4L[i,:,:], pdf_filter_test[i,-1,:,:])))))

list_bhatta_surf = np.zeros(pdf_filter_test.shape[0])
for i in range(pdf_filter_test.shape[0]):
    list_bhatta_surf[i] = 1 - np.sum(np.sqrt(np.abs((np.multiply(super_model_surf[i,:,:], pdf_filter_test[i,-1,:,:])))))



# Compute mass center and entropy

In [14]:
list_mc = np.zeros(pdf_filter_test.shape[0])
for i in range(pdf_filter_test.shape[0]):
    list_mc[i] = masse_center_mov(pdf_filter_test[i,-1,:,:])

list_entropie = np.zeros(pdf_filter_test.shape[0])
for i in range(pdf_filter_test.shape[0]):
#for i in range(10):
    list_entropie[i] = entropie(pdf_filter_test[i,-1,:,:])

# EKE and KE

In [15]:
folder = '/home/datawork-lemar-apero/tpicard/EKE/'
name_nc = 'MKE_EKE_box1040km_windows30j_sample20.nc'
file = folder+name_nc
nc = nc4.Dataset(file,'r')
EKE = np.asfortranarray(nc.variables['EKE'][:,:])
MKE = np.asfortranarray(nc.variables['MKE'][:,:])
date_EKE = np.asfortranarray(nc.variables['time'][:])
nc.close()


In [16]:
level_600 = 12 
level_200 = 4

window = 60
def image_selector_1layer(pos,var):
    pos_j = pos%6
    pos_i = pos//6
    d_ic = ic_all_list[pos_i]
    d_jc = jc_all_list[pos_j]
    var = var[window+d_ic:-window+d_ic,window+d_jc:-window+d_jc]

    return(var)

#for i in range(190):
EKE_mean = np.nanmean(EKE[:,level_200:,:,:],axis=1)
MKE_mean = np.nanmean(MKE[:,level_200:,:,:],axis=1)

EKE_pos = []

for i in range(EKE.shape[0]):
    for pos in range(0,36):
        KE_i = EKE_mean[i,:,:]
        KE_box = image_selector_1layer(pos,KE_i)
        EKE_pos.append(KE_box)
EKE_pos = np.array(EKE_pos)

#for i in range(190):
MKE_pos = []

for i in range(EKE.shape[0]):
    for pos in range(0,36):
        KE_i = MKE_mean[i,:,:]
        KE_box = image_selector_1layer(pos,KE_i)
        MKE_pos.append(KE_box)
        
MKE_pos = np.array(MKE_pos)
MKE_pos_cinquieme = MKE_pos[::5,:,:]
EKE_pos_cinquieme = EKE_pos[::5,:,:]
MKE_pos_cinquieme = np.nanmean(MKE_pos_cinquieme[list_rand,100:-100,100:-100],axis=(1,2))
EKE_pos_cinquieme = np.nanmean(EKE_pos_cinquieme[list_rand,100:-100,100:-100],axis=(1,2))


<ipython-input-16-fd5ed396fa16>:15: RuntimeWarning: Mean of empty slice
  EKE_mean = np.nanmean(EKE[:,level_200:,:,:],axis=1)
<ipython-input-16-fd5ed396fa16>:16: RuntimeWarning: Mean of empty slice
  MKE_mean = np.nanmean(MKE[:,level_200:,:,:],axis=1)


In [17]:
import pandas as pd

df = pd.DataFrame(

    {

        #"index": list_rand,

        "$BL_{200m}^{surf}$": list_bhatta_surf,
        
        "$BL_{200m}^{4L}$": list_bhatta,
        
        "$Mass$ $center$ $[m]$": list_mc,
        
        "$Entropy$": list_entropie,
        
        "$MKE$": MKE_pos_cinquieme*10000,
        
        "$EKE$": EKE_pos_cinquieme*10000,
        
        "$\zeta/f$": vrt_mean*100,
        
        "$OW/f$": ow_mean*1000

    }

)

In [18]:
df.set_index([pd.Index(np.linspace(1,26,26).astype(int))])

,$BL_{200m}^{surf}$,$BL_{200m}^{4L}$,$Mass$ $center$ $[m]$,$Entropy$,$MKE$,$EKE$,$\zeta/f$,$OW/f$
1,0.065862,0.045382,136.162468,6.498593,90.161644,81.087410,-4.575654,7.353759
2,0.273013,0.241548,135.925700,6.870018,94.393936,65.209343,-2.448304,1.951981
3,0.060084,0.092933,110.301224,6.383712,101.234047,81.522095,4.228233,-1.118412
4,0.023324,0.009708,63.255381,5.672064,124.491585,30.544758,-4.858424,-2.563969
5,0.186093,0.126889,183.422664,6.092232,126.787216,46.118259,0.860743,3.961301
6,0.124383,0.099249,216.040693,6.376312,107.295647,49.957703,-0.390359,-0.681547
7,0.194212,0.104023,109.279394,6.014067,96.037918,44.335518,-4.422597,-1.992488
8,0.170551,0.073051,164.263186,5.882004,98.047882,56.597980,0.758099,3.319996
9,0.082434,0.053390,172.298971,5.953147,81.991547,39.126766,2.992471,7.935832
10,0.115997,0.072408,178.026682,6.301736,78.742554,47.426525,3.220376,1.304982


In [25]:
#df.to_csv('table_25_exemples.cvs')
df.to_excel('table_25_exemples.xlsx')